In [1]:
#P13 In-Class 15481075 이재근
# 필요한 feature들을 추출하여 테스트 모델을 생성하고 비교하는 과정을 진행하였다. 타이타닉 csv파일을 이용하여 생존과 관련된 예측모델을 만들면서 모델에게 학습시킬 features를 골라 학습이 원할하게 진행될 수 있도록 알맞게 전처리 과정을 진행했다. 데이터 전처리 과정에서 train과 test데이터를 같은 방법으로 한 번에 처리하는 과정에서 두 개의 데이터를 합치는 과정도 진행했다. null값을 처리하는 과정도 간단한 하게 진행하는 것이 신기했고 더 많은 예측 모델을 만들어 비교해보는 과정도 공부해봐야겠다는 생각이 들었습니다.

In [2]:
import findspark

In [3]:
findspark.init('C:\spark-2.4.5-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('my_example').getOrCreate()

In [6]:
data = spark.read.csv('titanic.csv',inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [9]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [10]:
my_final_data = my_cols.na.drop()

In [11]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [12]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [13]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [14]:
assembler = VectorAssembler(inputCols = ['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
from pyspark.ml import Pipeline

In [17]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [18]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [19]:
train_titanic_data,test_titanic_data = my_final_data.randomSplit([0.7,0.3])

In [20]:
fit_model = pipeline.fit(train_titanic_data)

In [21]:
result = fit_model.transform(test_titanic_data)

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [24]:
result.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [25]:
ACC = my_eval.evaluate(result)

In [26]:
ACC

0.8162837162837162